In [63]:
import pandas as pd
import numpy as np
import pyodbc
import math
from ast import literal_eval
from tqdm import tqdm
import os
from configparser import ConfigParser
import statistics
from scipy import stats as s

In [64]:
pd.set_option("display.max_column",200)
pd.set_option("display.float_format",lambda x:'%.5f' %x)

In [66]:
config = ConfigParser()
config.read(r'/notebooks/notebooks/config.ini')
connection_string = config.get('SERVER', 'connection')
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

In [67]:
connection_string

'DRIVER=FreeTDS;SERVER=172.16.34.15;PORT=1433;DATABASE=FINCORE_DB;UID=Fincore_Analytics_USR;PWD=F!NCore@123;TDS_Version=7.2;'

In [5]:
cursor.execute('''INSERT INTO  FINCORE_DB.Fincore_Analytics.RISK_PERSON_ANALYTICAL_RULES
select
A.PERSON_MASTER_ID,
A.RULE_NO,
case 
	when c.THRESHOLD1_1 IS null then 0
	when A.RULE_VALUE < c.threshold1_1 then 0 
	when A.RULE_VALUE = c.threshold1_1 then 1
	when A.RULE_VALUE <= c.THRESHOLD1_2 then 2
	when A.RULE_VALUE <= c.THRESHOLD1_3 then 3
	when A.RULE_VALUE <= c.THRESHOLD1_4 then 4
	when A.RULE_VALUE <= c.THRESHOLD1_5 then 5
	when A.RULE_VALUE <= c.THRESHOLD1_6 then 6
	when A.RULE_VALUE <= c.THRESHOLD1_7 then 7
	when A.RULE_VALUE <= c.THRESHOLD1_8 then 8
	when A.RULE_VALUE <= c.THRESHOLD1_9 then 9
	when A.RULE_VALUE <= c.THRESHOLD1_10 then 10
	else 10
	end Risk_score,
	GETDATE() AS CREATE_DATE
from FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG_rules a 
inner join FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG b 
on a.PERSON_MASTER_ID = b.PERSON_MASTER_ID
inner join FINCORE_DB.Fincore_Analytics.RISK_lkp_entity_threshold c on 
c.PEER_GROUP_ID = b.PEER_GROUP_ID and a.RULE_NO = c.RULE_NO
where a.RISK_SCORE_STATIC is null''')
conn.commit()

IntegrityError: ('23000', "[23000] [FreeTDS][SQL Server]Violation of PRIMARY KEY constraint 'XPK_RISK_PERSON_ANALYTICAL_RULES'. Cannot insert duplicate key in object 'Fincore_Analytics.RISK_PERSON_ANALYTICAL_RULES'. The duplicate key value is (1, P002001001). (2627) (SQLExecDirectW)")

In [68]:

WEIGHTED_AVG = pd.read_sql_query('''
SELECT PERSON_MASTER_ID,
PRIORITY_NO,
CASE WHEN SUM(THRESHOLD_FLG)!= 0 THEN SUM(SQUARE(THRESHOLD_FLG))/SUM(THRESHOLD_FLG) 
ELSE 0
END AS FINAL
FROM (
SELECT T.*,PRIORITY_NO FROM FINCORE_DB.FINCORE_ANALYTICS.RISK_LKP_RULES RLR WITH (NOLOCK)
INNER JOIN 
(
SELECT DISTINCT PERSON_MASTER_ID,RULE_NO,RISK_SCORE_STATIC AS THRESHOLD_FLG 
FROM FINCORE_DB.FINCORE_ANALYTICS.RISK_PERSON_SUMMARY_STG_RULES
WHERE RISK_SCORE_STATIC IS NOT NULL

UNION 

SELECT DISTINCT
A.PERSON_MASTER_ID,
A.RULE_NO,
coalesce((CASE 
	WHEN C.THRESHOLD1_1 IS NULL THEN 0
	WHEN A.RULE_VALUE < C.THRESHOLD1_1 THEN 0 
	WHEN A.RULE_VALUE = C.THRESHOLD1_1 THEN 1
	WHEN A.RULE_VALUE <= C.THRESHOLD1_2 THEN 2
	WHEN A.RULE_VALUE <= C.THRESHOLD1_3 THEN 3
	WHEN A.RULE_VALUE <= C.THRESHOLD1_4 THEN 4
	WHEN A.RULE_VALUE <= C.THRESHOLD1_5 THEN 5
	WHEN A.RULE_VALUE <= C.THRESHOLD1_6 THEN 6
	WHEN A.RULE_VALUE <= C.THRESHOLD1_7 THEN 7
	WHEN A.RULE_VALUE <= C.THRESHOLD1_8 THEN 8
	WHEN A.RULE_VALUE <= C.THRESHOLD1_9 THEN 9
	WHEN A.RULE_VALUE <= C.THRESHOLD1_10 THEN 10
	ELSE 10
	END),1) THRESHOLD_FLG
	FROM FINCORE_DB.FINCORE_ANALYTICS.RISK_PERSON_SUMMARY_STG_RULES A 
INNER JOIN FINCORE_DB.FINCORE_ANALYTICS.RISK_PERSON_SUMMARY_STG B ON A.PERSON_MASTER_ID = B.PERSON_MASTER_ID
INNER JOIN FINCORE_DB.FINCORE_ANALYTICS.RISK_LKP_ENTITY_THRESHOLD C ON C.PEER_GROUP_ID = B.PEER_GROUP_ID AND A.RULE_NO = C.RULE_NO
INNER JOIN FINCORE_DB.FINCORE_ANALYTICS.RISK_LKP_RULES D ON D.RULE_NO = A.RULE_NO

WHERE A.RISK_SCORE_STATIC IS NULL
)T
ON T.RULE_NO = RLR.RULE_NO

)T
GROUP BY PERSON_MASTER_ID,PRIORITY_NO
''',conn)

In [69]:
weights = pd.read_sql_query('''Select distinct PRIORITY_NO, WEIGHTAGE as WEIGHTS FROM FINCORE_DB.FINCORE_ANALYTICS.RISK_LKP_RULES WITH (NOLOCK)''',conn)

In [70]:
weights.dropna(inplace = True)

In [58]:
WEIGHTED_AVG = WEIGHTED_AVG.merge(weights,how = 'inner',on = 'PRIORITY_NO')

In [59]:
WEIGHTED_AVG['WEIGHTED_SCORE'] = WEIGHTED_AVG['FINAL']*WEIGHTED_AVG['WEIGHTS']

In [15]:
WEIGHTED_AVG.drop(columns = ['PRIORITY_NO','FINAL'],inplace = True)

In [17]:
FINAL_DF = WEIGHTED_AVG.groupby(['PERSON_MASTER_ID']).sum()

In [18]:
FINAL_DF.reset_index(inplace = True)

In [20]:
risk_person = pd.read_sql_query('''Select PERSON_MASTER_ID,PEER_GROUP_ID from fincore_db.fincore_analytics.risk_person_summary_stg''',conn)

In [21]:
risk_person

PERSON_MASTER_ID        PEER_GROUP_ID
0                   1  Government Sector_0
1                   2  Government Sector_0
2                   3  Government Sector_0
3                   4  Government Sector_0
4                   5  Government Sector_0
..                ...                  ...
424               536     OTHERS_CHANGED_9
425               537             Others_0
426               538             Others_0
427               539             Others_0
428               540             Others_0

[429 rows x 2 columns]

In [22]:
FINAL_DF

PERSON_MASTER_ID  WEIGHTS  WEIGHTED_SCORE
0                   1  0.95000         6.73071
1                   2  0.95000         7.40000
2                   3  0.95000         7.40000
3                   4  0.95000         7.40000
4                   5  0.95000         7.40000
..                ...      ...             ...
423               536  0.95000         7.06058
424               537  0.30000         0.00000
425               538  0.30000         0.00000
426               539  0.45000         0.00000
427               540  0.30000         0.00000

[428 rows x 3 columns]

In [23]:
auto_high_df = pd.read_sql_query('''SELECT PERSON_MASTER_ID FROM FINCORE_DB.FINCORE_ANALYTICS.RISK_PERSON_SUMMARY_STG_RULES RTSSR WITH(NOLOCK)
INNER JOIN FINCORE_DB.FINCORE_ANALYTICS.RISK_LKP_RULES RLR WITH (NOLOCK) ON RLR.RULE_NO = RTSSR.RULE_NO 
WHERE RISK_TYPE = 'AUTO HIGH RULES '
''',conn)

In [24]:
auto_high_person = auto_high_df['PERSON_MASTER_ID'].to_list()

In [25]:
auto_high_person

[5, 46, 51, 147]

In [26]:
FINAL_DF = FINAL_DF[~FINAL_DF['PERSON_MASTER_ID'].isin(auto_high_person)]

In [27]:
FINAL_DF = FINAL_DF.merge(risk_person,how = 'inner', on = 'PERSON_MASTER_ID')

In [28]:
FINAL_DF['RISK_SCORE'] = FINAL_DF['WEIGHTED_SCORE']/FINAL_DF['WEIGHTS']

In [30]:
FINAL_DF['RISK_SCORE'] = FINAL_DF['RISK_SCORE'].apply(lambda x: round(x))

In [31]:
FINAL_DF

PERSON_MASTER_ID  WEIGHTS  WEIGHTED_SCORE        PEER_GROUP_ID  \
0                   1  0.95000         6.73071  Government Sector_0   
1                   2  0.95000         7.40000  Government Sector_0   
2                   3  0.95000         7.40000  Government Sector_0   
3                   4  0.95000         7.40000  Government Sector_0   
4                   6  0.95000         7.40556     OTHERS_CHANGED_0   
..                ...      ...             ...                  ...   
419               536  0.95000         7.06058     OTHERS_CHANGED_9   
420               537  0.30000         0.00000             Others_0   
421               538  0.30000         0.00000             Others_0   
422               539  0.45000         0.00000             Others_0   
423               540  0.30000         0.00000             Others_0   

     RISK_SCORE  
0             7  
1             8  
2             8  
3             8  
4             8  
..          ...  
419           7  
420           0  
421           0  
422           0  
423           0  

[424 rows x 5 columns]

In [32]:
count = 1
#UPDATING RISK_PERSON
for ind,row in tqdm(FINAL_DF.iterrows()):
    if count%10000 == 0:
        print(row['PERSON_MASTER_ID'],' HERE')
        conn.commit()
    count += 1

    conn.execute('''insert into FINCORE_DB.Fincore_Analytics.RISK_PERSON (PERSON_MASTER_ID,PEER_GROUP_ID,RISK_SCORE,CREATE_DATE)
    values ({},'{}',{},{})'''.format(row['PERSON_MASTER_ID'],row['PEER_GROUP_ID'],row['RISK_SCORE'],'Getdate()'))

conn.commit()


0it [00:00, ?it/s]


IntegrityError: ('23000', "[23000] [FreeTDS][SQL Server]Violation of PRIMARY KEY constraint 'XPK_RISK_PERSON'. Cannot insert duplicate key in object 'Fincore_Analytics.RISK_PERSON'. The duplicate key value is (1). (2627) (SQLExecDirectW)")

In [33]:
cursor.execute('''Update fincore_db.fincore_analytics.risk_person set risk_score = 1 where risk_score = 0''')
conn.commit()
conn.close()

In [ ]:
%store -r

In [35]:
%store -r no_of_records
print('no_of_records',no_of_records)
%store -r job_id
print('job_id',job_id)

no stored variable or alias no_of_records


In [39]:
config = ConfigParser()
config.read(r'/notebooks/notebooks/config.ini')
connection_string = config.get('SERVER', 'connection')
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

In [40]:
cursor.execute(f'''Update FINCORE_DB.Fincore_Analytics.AUDIT_INTGN set JOB_END_DATE = Getdate(),JOB_STATUS = 'SUCCESS',NO_OF_RECORD = {no_of_records} where  JOB_RUN_ID ={job_id}''')
cursor.commit()
conn.close()

NameError: name 'no_of_records' is not defined

In [ ]:
print('Individual Risk Completed.')